In [2]:
import csv
import numpy as np
import logging
import matplotlib.pyplot as plt
from imp import reload
reload(logging)

<module 'logging' from 'D:\\Anaconda3\\lib\\logging\\__init__.py'>

# More Statistics

The code below is the same as the code in the 'anomaly_global' notebook, except that now we store the minimum and maximum transaction value for each account in 'min_dict' and 'max_dixt', as well as the ratio between the current value and the historical minimum in the variable 'ratio'. The ratio can be used for further actions, but here I only extract this quantity without using it.

In [4]:
%%capture

reload(logging)
logging.basicConfig(filename='anomaly_global_final.log', filemode='w', level=logging.DEBUG)
with open('task_data_small.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    
    merchant_dict = {} # store the merchant ID under each account
    amount_dict = {} # store the transaction amount for corresponding merchant ID under each account
    mean_dict = {} # store the moving average under each account
    std_dict = {} # store the moving standard deviation under each account
    anomaly_dict = {} # store the index of anomaly transactions under each account
    account_count_dict = {} # count the number of uncorrupted transactions under rach account
    mean_global = 0
    std_global = 0
    amount_global = [] # store the uncorrupted transactions for all customers
    
    min_dict = {} # store the current min value for an account
    max_dict = {} # store the current max value for an account
    
    anomaly_count = 0 # count the total number of anomalies
    
    for row in csv_reader:
        print("====================")
        
        # first ensure only the data that is not corrupted is processed
        if len(row)==3 and row[0]!='' and row[1]!='' and row[2]!='' and row[0]!='X' and row[1]!='X' and row[2]!='X': 
            account = row[0].strip() # account ID
            merchant = row[1].strip() # merchant ID
            amount = float(row[2]) # transaction amount
            print("Account: "+account)
            print("Merchant: "+merchant)
            print("Transaction amount: "+row[2])
            
            # obtain the global statistics
            amount_global.append(amount)
            mean_global = np.mean(amount_global)
            std_global = np.std(amount_global)
            
            # current account appeared before
            if account in merchant_dict:
                current_mean = mean_dict[account][-1]
                current_std = std_dict[account][-1]
                diff = amount - current_mean
                diff_global = amount - mean_global
                
                account_count_dict[account] += 1
                
                merchant_dict[account].append(merchant)
                amount_dict[account].append(amount)
                new_mean = np.mean(amount_dict[account])
                new_std = np.std(amount_dict[account])
                mean_dict[account].append(new_mean)
                std_dict[account].append(new_std)
                
                # track the min and max values
                new_min = min(amount_dict[account])
                new_max = max(amount_dict[account])
                min_dict[account] = new_min
                max_dict[account] = new_max
                
                ratio = amount / new_min # the ratio between the current value and the historical minimum, which can be used for further actions
                print("Mean of transaction amount so far: "+str(new_mean))
                print("Std of transaction amount so far: "+str(new_std))
                                
                # if there are in total less than 3 transactions globally
                if len(amount_global)<=2:
                    print("Status: normal")
                
                # if there are less than 4 transactions for this account
                elif len(amount_dict[account])<=3:
                    if (diff>=-3*current_std and diff<=3*current_std)or(diff_global>=-3*std_global and diff_global<=3*std_global):
                        print("Status: normal")
                    else:
                        anomaly_count += 1
                        if account in anomaly_dict:
                            anomaly_dict[account].append(account_count_dict[account])
                        else:
                            anomaly_dict[account]=[account_count_dict[account]]

                        logging.warning('Flag: '+'account: '+account+' merchant: '+merchant+' amount: '+str(amount))
                        print("Status: abnormal. The event has been recorded in log file.")
                
                # if there are more than 3 transactions for this account
                else:
                    if diff>=-3*current_std and diff<=3*current_std:
                        print("Status: normal")
                    else:
                        anomaly_count += 1
                        if account in anomaly_dict:
                            anomaly_dict[account].append(account_count_dict[account])
                        else:
                            anomaly_dict[account]=[account_count_dict[account]]

                        logging.warning('Flag: '+'account: '+account+' merchant: '+merchant+' amount: '+str(amount))
                        print("Status: abnormal. The event has been recorded in log file.")
                    
            
            # first time seeing the current account
            else:
                print("Mean of transaction amount so far: "+str(amount))
                print("Std of transaction amount so far: 0")
        
                merchant_dict[account] = [merchant]
                amount_dict[account] = [amount]
                mean_dict[account] = [amount]
                std_dict[account] = [0]
                min_dict[account] = [amount]
                max_dict[account] = [amount]
                
                ratio = 1.
                
                diff_global = amount - mean_global
                
                account_count_dict[account]=0
                
                if len(amount_global)<=2:
                    print("Status: normal")
                elif diff_global>=-3*std_global and diff_global<=3*std_global:
                    print("Status: normal")
                else:
                    anomaly_count += 1
                    if account in anomaly_dict:
                        anomaly_dict[account].append(account_count_dict[account])
                    else:
                        anomaly_dict[account]=[account_count_dict[account]]
                        
                    logging.warning('Flag: '+'account: '+account+' merchant: '+merchant+' amount: '+str(amount))
                    print("Status: abnormal. The event has been recorded in log file.")
                    
        # corrupted data
        else:
            account = row[0].strip() # account ID
            merchant = row[1].strip() # merchant ID
            print("Account: "+account)
            print("Merchant: "+merchant)
            print('The data of this transaction is corrupted' )